In [1]:
from model import Models
from sklearn.metrics import  r2_score,mean_squared_error
import pandas as pd
import preprocessing
    
df_train=pd.read_csv('./data/airbnb_train.csv', sep=";",decimal='.' )
df_test=pd.read_csv('./data/airbnb_test.csv', sep=";",decimal='.' )

df_train=df_train.dropna(subset=['Price'])
df_test=df_test.dropna(subset=['Price'])
df_train=preprocessing.filtrar_columnas(df_train,['Cleaning Fee','Accommodates','Room Type','Host Listings Count','Bathrooms','Bedrooms','Guests Included','Price'])
df_train , imputer , xnumeric = preprocessing.imputer_data_numeric_2(df_train)
df_train , encoding= preprocessing.encoding_categorical_column_2(df_train)

In [2]:
hf = df_train.isnull().sum()
columns_with_null= hf[hf > 0]
columns_with_null

Series([], dtype: int64)

In [3]:
df_test = df_test[df_train.columns.intersection(df_test.columns)]
df_test[xnumeric] = imputer.transform(df_test[xnumeric])

In [4]:
df_test.columns

Index(['Cleaning Fee', 'Accommodates', 'Room Type', 'Host Listings Count',
       'Bathrooms', 'Bedrooms', 'Guests Included', 'Price'],
      dtype='object')

In [5]:
hf = df_test.isnull().sum()
columns_with_null= hf[hf > 0]
columns_with_null

Series([], dtype: int64)

In [6]:
preprocessing.save_label_encoder(encoding)

In [7]:
for col, encoder_info in encoding.items():
    if encoder_info['type'] == 'LabelEncoder':
        df_test[col] = encoder_info['model'].transform(df_test[col].astype(str)) if col in encoder_info['model'].classes_ else -1
      

In [8]:
X_train = df_train.drop(columns=["Price"]) 
y_train = df_train["Price"] 
X_test = df_test.drop(columns=["Price"])
y_test = df_test["Price"]

In [ ]:
model=Models()
model.y_test=y_test
model.X_test=X_test
model.grid_training(X_train,y_train)
bestm=model.get_best_model()
y_predict=bestm.predict(X_test)
test_r2 = r2_score(y_test, y_predict)
test_mse = mean_squared_error(y_test, y_predict)
print(f"Test R^2: {test_r2:.4f}")
print(f"Test MSE: {test_mse:.4f}")